# DAP 4

# Imports

## Hyper-Parameter Finding
### Multivariate Linear Regression Model

### Multivariate Linear Regression Model with Gradient Descent

### Regularized Regression Model with Feature Normalization

### Polynomial Regression Model

## Finding Optimal Size of Training Data

## Model Evaluation